In [27]:
import pandas as pd
import locale
import glob,os, random
import os.path
import requests
import tarfile


# Convert text to lower-case and strip punctuation/symbols from words
def normalize_text(text):
    norm_text = text.lower()

    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', ' ')

    # Pad punctuation with spaces on both sides
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        norm_text = norm_text.replace(char, ' ' + char + ' ')

    return norm_text

In [28]:
fields = ['foldername', 'label']


df = pd.read_csv('/scratch/qx344/speech_data/presidents_meta.csv', usecols=fields,skipinitialspace=True)
# See the keys
print df.keys(),len(df.index)

Index([u'foldername', u'label'], dtype='object') 34


In [29]:
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple
import io

SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')

alldocs = []  # will hold all docs in original order

#for line_no, line in enumerate(alldata):
index = 0
for line_no, row in df.iterrows():
    sentiment = row['label']*1.0
    os.chdir("/scratch/qx344/speech_data/data/{0}".format(row['foldername']))
    filesNum = len(os.listdir('.'))
    testfile = random.choice(os.listdir('.')) 
    with open(testfile, 'r') as myfile:
        line = myfile.read().replace('\n', '')
        words = gensim.utils.to_unicode(line).split()
        split ='test' 
        tags = [index] # `tags = [tokens[0]]` would also work at extra memory cost
        index = index + 1
        alldocs.append(SentimentDocument(words, tags, split, sentiment))
        trainfilelist = glob.glob("*.txt")
        trainfilelist.remove(testfile)
    for txtfile in trainfilelist:
        with open(txtfile, 'r') as myfile:
            line = myfile.read().replace('\n', '')
            words = gensim.utils.to_unicode(line).split()
            tags = [index] # `tags = [tokens[0]]` would also work at extra memory cost
            index = index + 1
            split ='train'  
            alldocs.append(SentimentDocument(words, tags, split, sentiment))

train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']
doc_list = alldocs[:]  # for reshuffling per pass

print('%d docs: %d train-sentiment, %d test-sentiment' % (len(doc_list), len(train_docs), len(test_docs)))

862 docs: 828 train-sentiment, 34 test-sentiment


In [31]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

simple_models = [
    # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, size=80, window=5, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DBOW 
    Doc2Vec(dm=0, size=80, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=80, window=10, negative=5, hs=0, min_count=2, workers=cores),
]

# speed setup by sharing results of 1st model's vocabulary scan
simple_models[0].build_vocab(alldocs)  # PV-DM/concat requires one special NULL word so it serves as template
print(simple_models[0])
for model in simple_models[1:]:
    model.reset_from(simple_models[0])
    print(model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

Doc2Vec(dm/c,d80,n5,w5,mc2,t20)
Doc2Vec(dbow,d80,n5,mc2,t20)
Doc2Vec(dm/m,d80,n5,w10,mc2,t20)


In [32]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[2]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[0]])

In [33]:
import numpy as np
import statsmodels.api as sm
from random import sample

# for timing
from contextlib import contextmanager
from timeit import default_timer
import time 

@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    
    
    
    
from sklearn import linear_model


def error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in train_set])
    #train_regressors = sm.add_constant(train_regressors)
    logit = linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, 
                                        intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=300, 
                                        multi_class='multinomial', verbose=0, warm_start=False, n_jobs=1)
    predictor = logit.fit(train_regressors,train_targets)

    test_targets, test_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in test_set])
    
    corrects = predictor.score(test_regressors,test_targets)
    error_rate= 1 - corrects
    errors = len(test_targets)*error_rate
    return (error_rate, errors, len(test_targets), predictor)

In [34]:
from collections import defaultdict
best_error = defaultdict(lambda :1.0)  # to selectively-print only best errors achieved

In [35]:
from random import shuffle
import datetime

alpha, min_alpha, passes = (0.025, 0.001, 20)
alpha_delta = (alpha - min_alpha) / passes

print("START %s" % datetime.datetime.now())

for epoch in range(passes):
    shuffle(doc_list)  # shuffling gets best results
    
    for name, train_model in models_by_name.items():
        # train
        duration = 'na'
        train_model.alpha, train_model.min_alpha = alpha, alpha
        with elapsed_timer() as elapsed:
            train_model.train(doc_list)
            duration = '%.1f' % elapsed()
            
        # evaluate
        eval_duration = ''
        with elapsed_timer() as eval_elapsed:
            err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs)
        eval_duration = '%.1f' % eval_elapsed()
        best_indicator = ' '
        if err <= best_error[name]:
            best_error[name] = err
            best_indicator = '*' 
        print("%s%f : %i passes : %s %ss %ss" % (best_indicator, err, epoch + 1, name, duration, eval_duration))

#         if ((epoch + 1) % 5) == 0 or epoch == 0:
#             eval_duration = ''
#             with elapsed_timer() as eval_elapsed:
#                 infer_err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs, infer=True)
#             eval_duration = '%.1f' % eval_elapsed()
#             best_indicator = ' '
#             if infer_err < best_error[name + '_inferred']:
#                 best_error[name + '_inferred'] = infer_err
#                 best_indicator = '*'
#             print("%s%f : %i passes : %s %ss %ss" % (best_indicator, infer_err, epoch + 1, name + '_inferred', duration, eval_duration))

    print('completed pass %i at alpha %f' % (epoch + 1, alpha))
    alpha -= alpha_delta
    
print("END %s" % str(datetime.datetime.now()))

START 2016-12-04 21:21:46.252534
*0.529412 : 1 passes : Doc2Vec(dm/c,d80,n5,w5,mc2,t20) 19.6s 0.1s
*0.294118 : 1 passes : Doc2Vec(dbow,d80,n5,mc2,t20) 6.0s 0.1s
*0.558824 : 1 passes : Doc2Vec(dm/m,d80,n5,w10,mc2,t20) 10.7s 0.2s
*0.352941 : 1 passes : dbow+dmm 0.0s 0.2s
*0.382353 : 1 passes : dbow+dmc 0.0s 0.2s
completed pass 1 at alpha 0.025000
*0.441176 : 2 passes : Doc2Vec(dm/c,d80,n5,w5,mc2,t20) 17.1s 0.1s
*0.294118 : 2 passes : Doc2Vec(dbow,d80,n5,mc2,t20) 6.0s 0.1s
*0.352941 : 2 passes : Doc2Vec(dm/m,d80,n5,w10,mc2,t20) 10.7s 0.2s
 0.382353 : 2 passes : dbow+dmm 0.0s 0.2s
*0.294118 : 2 passes : dbow+dmc 0.0s 0.2s
completed pass 2 at alpha 0.023800
 0.529412 : 3 passes : Doc2Vec(dm/c,d80,n5,w5,mc2,t20) 16.2s 0.1s
 0.411765 : 3 passes : Doc2Vec(dbow,d80,n5,mc2,t20) 6.0s 0.1s
 0.382353 : 3 passes : Doc2Vec(dm/m,d80,n5,w10,mc2,t20) 10.5s 0.2s
 0.382353 : 3 passes : dbow+dmm 0.0s 0.2s
 0.323529 : 3 passes : dbow+dmc 0.0s 0.2s
completed pass 3 at alpha 0.022600
*0.441176 : 4 passes : Do

In [ ]:
##### please omit anything below ############

In [ ]:
##### please omit anything below ############

In [ ]:
##### please omit anything below ############

In [7]:
from random import shuffle
import datetime

alpha, min_alpha, passes = (0.025, 0.001, 20)
alpha_delta = (alpha - min_alpha) / passes


shuffle(doc_list)  # shuffling gets best results

limiter = 1
for name, train_model in models_by_name.items():
    # train
    train_model.alpha, train_model.min_alpha = alpha, alpha
    train_model.train(doc_list)
    if limiter>0:
        break


In [13]:
err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs)

In [75]:
train_targets, train_regressors = zip(*[(doc.sentiment, train_model.docvecs[doc.tags[0]]) for doc in train_docs])

In [76]:
test_targets, test_regressors = zip(*[(doc.sentiment, train_model.docvecs[doc.tags[0]]) for doc in test_docs])

In [77]:
from sklearn import linear_model

classfier1 =linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, 
                                        intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=300, 
                                        multi_class='multinomial', verbose=0, warm_start=False, n_jobs=1)

In [78]:
classfier1.fit(train_regressors,train_targets)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=300, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [79]:
classfier1.score(test_regressors,test_targets)

0.43021999999999999

In [80]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
classfier2 =  OneVsRestClassifier(LinearSVC(random_state=0))

In [81]:
classfier2.fit(train_regressors,train_targets)

OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [82]:
classfier2.score(test_regressors,test_targets)

0.43181999999999998

In [68]:
from sklearn import svm
classfier3 = svm.SVC()
classfier3.fit(train_regressors,train_targets) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [69]:
classfier3.score(test_regressors,test_targets)

0.313

In [ ]:
logit = sm.Logit(train_targets, train_regressors)

error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1)
    
error_rate_for_model(train_model, train_docs, test_docs)

In [ ]:
train_targets, train_regressors = zip(*[(doc.sentiment, train_model.docvecs[doc.tags[0]]) for doc in train_docs])
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    #test_data = test_set
    test_data = test_set  ### for test
    if infer:
        if infer_subsample < 1.0:
            test_data = sample(test_data, int(infer_subsample * len(test_data)))
        test_regressors = [test_model.infer_vector(doc.words, steps=infer_steps, alpha=infer_alpha) for doc in test_data]
    else:
        #print test_docs[:3]  ##  for test
        test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_docs]
    test_regressors = sm.add_constant(test_regressors)
    
    # predict & evaluate
    Rawtest_predictions = predictor.predict(test_regressors)
    
    #print test_predictions ## for test
    test_predictions = Rawtest_predictions #[ np.argmax(RL) for RL in Rawtest_predictions] ## for test
    #print test_predictions ## for test
    
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_data])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

In [ ]:
import glob, os
os.chdir("/scratch/qx344/speech_data/data/{0}".format(df['foldername'][0]))
for file in glob.glob("*.txt"):
    print(file)

In [ ]:
print df['foldername'][0]
os.chdir("/scratch/qx344/speech_data/data/{0}".format(df['foldername'][0]))
len(os.listdir('.')) 

In [ ]:
with open('/scratch/qx344/speech_data/data/adams/combined.txt', 'r') as myfile:
    data=myfile.read().replace('\n', '')

In [14]:
### this is picking last file as text file
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple
import io

SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')

alldocs = []  # will hold all docs in original order

#for line_no, line in enumerate(alldata):
index = 0
for line_no, row in df.iterrows():
    sentiment = row['label']*1.0
    os.chdir("/scratch/qx344/speech_data/data/{0}".format(row['foldername']))
    filesNum = len(os.listdir('.'))
    localindex = 1
    for txtfile in glob.glob("*.txt"):
        with open(txtfile, 'r') as myfile:
            line = myfile.read().replace('\n', '')
            words = gensim.utils.to_unicode(line).split()
            tags = [index] # `tags = [tokens[0]]` would also work at extra memory cost
            index = index + 1
            split ='train' if localindex<filesNum else 'test' #['train','train','train','test'][line_no//25000]  # 75k train, 25k test
            localindex = localindex+1
            alldocs.append(SentimentDocument(words, tags, split, sentiment))

train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']
doc_list = alldocs[:]  # for reshuffling per pass

print('%d docs: %d train-sentiment, %d test-sentiment' % (len(doc_list), len(train_docs), len(test_docs)))

22 docs: 21 train-sentiment, 1 test-sentiment
